### 本助手包含了以下功能：
-  查询天气
-  获取当前时间
-  通用知识问答

In [122]:

from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")

online_llm = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)




In [123]:
#导入建立 agent 的相关环境
from langchain.agents import initialize_agent, AgentType
from langchain.agents import Tool #定义工具信息的
from pydantic import BaseModel,Field
from langchain.tools import BaseTool
from typing import Type
import requests

In [124]:
AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION

<AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION: 'chat-conversational-react-description'>

In [125]:
# 聚合网相关api调用-根据城市查询天气 - 代码参考（根据实际业务情况修改）
# 基本参数配置
apiUrl = 'http://apis.juhe.cn/simpleWeather/query'  # 接口请求URL
juheAppKey = os.getenv('juheAppKey')  # 在个人中心->我的数据,接口名称上方查看

def get_weather(city:str):
    # 接口请求入参配置
    requestParams = {
        'key': juheAppKey,
        'city': city,
    }
  # 发起接口网络请求
    response = requests.get(apiUrl, params=requestParams)

    # 解析响应结果
    if response.status_code == 200:
        responseResult = response.json()
        # 网络请求成功。可依据业务逻辑和接口文档说明自行处理。
        print(responseResult)
        return responseResult
    else:
        # 网络异常等因素，解析结果异常。可依据业务逻辑自行处理。
        return '请求异常'

In [126]:
#定义 获取天气的 工具

class GetWeatherInput(BaseModel):
    city: str = Field(description="城市名称")

class GetWeather(BaseTool):
    name: str = Field(default="get_weather")
    description: str = Field(default="这是一个获取天气的API，可以获取指定城市的天气情况，需要用户提供城市名称")
    args_schema: Type[BaseModel] = GetWeatherInput

    def _run(self, city: str) -> str:
        print(f'开始查询{city}的天气信息')
        return get_weather(city)
    async def _arun(self, city: str) -> str:
        raise NotImplementedError("This tool does not support async")


In [127]:
#定义时间工具

import datetime

class GetDateInput(BaseModel):
    # format: str = Field(default="%Y-%m-%d", description="日期格式")
    pass

class GetDate(BaseTool):
    name: str = Field(default="get_date")
    description: str = Field(default="这是一个获取日期的API，可以获取当前日期")
    # args_schema: Type[BaseModel] = GetDateInput

    def _run(self) -> str:
        return datetime.datetime.now().strftime('%Y-%m-%d')
    async def _arun(self) -> str:
        raise NotImplementedError("This tool does not support async")


In [128]:
# 路由链相关环境
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.router import MultiRouteChain, LLMRouterChain, MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [129]:
# 底层 目标链的prompt
math_prompt = """
你是一位数学家，请根据以下问题，生成对应的数学公式.内容：{input}
"""

weather_prompt = """你是一位天气预报员，请根据以下问题，生成对应的天气信息.内容：{input}"""
time_prompt = """你是一位时间大师，请根据以下问题，获取对应时间.内容：{input}"""

In [130]:
#提示词模版 相关信息
prompt_info = [
    {
        "name": "math",
        "description": "数学问题",
        "prompt": math_prompt
    },
    {
        "name": "weather",
        "description": "天气问题",
        "prompt": weather_prompt
    },
    {
        "name": "time",
        "description": "时间问题",
        "prompt": time_prompt
    }
]

In [ ]:
#定义 目标链
math_LLMChain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(math_prompt))

#此单元格 为构建智能体原始方式，输出字段默认为 output
#定义 目标 智能体
weather_agent = initialize_agent(
    [GetWeather()],
    online_llm,
    AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
)

time_agent = initialize_agent(
    [GetDate()],
    online_llm,
    AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
)

In [ ]:
#修改 agent 智能体输出的字段, 此单元格方式虽然可以修改输出字段，但是在 multipromptChain 中无法生效，会报错
from langchain_core.runnables import  RunnableLambda

time_agent2 = initialize_agent(
    [GetDate()],
    online_llm,
    AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    # 关键配置：添加输出解析器
) | RunnableLambda(lambda x: {'input': x["input"], "text": x["output"]})


#定义 目标 智能体
weather_agent2 = initialize_agent(
    [GetWeather()],
    online_llm,
    AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
) | RunnableLambda(lambda x: {'input': x["input"], "text": x["output"]})


In [175]:
#initialize_agent智能体的返回字段为 output，目前尚未找到修改方法，故采用自定义 Chain来包裹智能体，变成chain的子类，从而修改返回字段

# 初始化 Agent 
time_agent_output = initialize_agent(
    [GetDate()],
    online_llm,
    AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

from langchain.chains.base import Chain
from langchain.agents import StructuredChatAgent
from typing import Dict, Any, List

#定义一个自定义的 Chain 类
class CustomChain(Chain):

    input_key: str = "input" #输入字段名
    output_key: str = "text" #输出字段名

    @property
    def input_keys(self) -> List[str]:
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        # 调用原始的 Agent
        # 从输入中获取用户的问题
        user_input = inputs[self.input_key]

        # 在这里，我们可以先做一些预处理，例如用另一个链处理输入

        # 然后调用agent

        agent_response = time_agent_output.run(user_input)

        # 这里可以对agent_response进行后处理

        # print(agent_response)

        return {self.output_key: agent_response}
    
    def run(self, *args, **kwargs):
        return self._call(*args, **kwargs)


# 使用自定义的 Chain
time_agent_text = CustomChain()
# 调用时直接获取 text 字段
result = time_agent_text.run({"input": "今天几号？"})
print(result)



> Entering new AgentExecutor chain...
Thought: 我需要使用get_date工具来获取当前日期。

Action:
```
{
  "action": "get_date",
  "action_input": {}
}
```

Observation: 2025-05-28
Thought:我现在知道日期是2025年5月28日。

Action:
```
{
  "action": "Final Answer",
  "action_input": "今天是2025年5月28日。"
}
```

> Finished chain.
{'text': '今天是2025年5月28日。'}


In [180]:
# 将 天气查询工具 也按照自定义的 Chain 来包装
# 初始化 Agent 

weather_agent_output = initialize_agent(
    [GetWeather()],
    online_llm,
    AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True, # 处理解析错误
)

from langchain.chains.base import Chain
from langchain.agents import StructuredChatAgent
from typing import Dict, Any, List

#定义一个自定义的 Chain 类
class CustomWeatherChain(Chain):

    input_key: str = "input" #输入字段名
    output_key: str = "text" #输出字段名

    @property
    def input_keys(self) -> List[str]:
        return [self.input_key]

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        # 调用原始的 Agent
        # 从输入中获取用户的问题
        user_input = inputs[self.input_key]

        # 在这里，我们可以先做一些预处理，例如用另一个链处理输入

        # 然后调用agent

        agent_response = weather_agent_output.run(user_input)

        # 这里可以对agent_response进行后处理

        # print(agent_response)

        return {self.output_key: agent_response}
    
    def run(self, *args, **kwargs):
        return self._call(*args, **kwargs)


# 使用自定义的 Chain
weather_agent_text = CustomWeatherChain()



> Entering new AgentExecutor chain...
Thought: 我需要使用get_date工具来获取当前日期。

Action:
```
{
  "action": "get_date",
  "action_input": {}
}
```

Observation: 2025-05-28
Thought:我现在知道日期了，可以直接回答问题。

Action:
```
{
  "action": "Final Answer",
  "action_input": "今天是2025年5月28日。"
}
```

> Finished chain.


{'input': '今天是几号？', 'output': '今天是2025年5月28日。'}



> Entering new AgentExecutor chain...
Thought: 我需要使用get_date工具来获取当前日期

Action:
```
{
  "action": "get_date",
  "action_input": {}
}
```

Observation: 2025-05-28
Thought:我现在知道今天的日期是2025年5月28日。

Action:
```
{
  "action": "Final Answer",
  "action_input": "今天是2025年5月28日。"
}
```

> Finished chain.


{'input': '今天是几号？', 'text': '今天是2025年5月28日。'}

In [135]:
#定义默认链提示词
default_prompt = """你是一位助手，别人无法回答的问题，你来回答，请回答以下问题：{input}"""

In [136]:
#定义默认链
default_chain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(template=default_prompt), output_key="text")

In [137]:
#路由配置
prompt_info = [
    "math:你是一位数学家，请根据以下问题，生成对应的数学公式",
    "weather:你是一位天气预报员，请根据以下问题，生成对应的天气信息",
    "time:你是一位时间大师，请根据以下问题，生成对应的时间信息",
]

In [138]:
#构建路由模版
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = "\n".join(prompt_info))
print("路由模版是：",router_template)

router_prompt = PromptTemplate(
    template=router_template,
    # template="\n".join(prompt_info),
    input_variables=["input"],
    output_parser=RouterOutputParser()
    )
print("路由模版router_prompt 是：",router_prompt)

路由模版是： Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
math:你是一位数学家，请根据以下问题，生成对应的数学公式
weather:你是一位天气预报员，请根据以下问题，生成对应的天气信息
time:你是一位时间大师，请根据以下问题，生成对应的

In [139]:
#构建路由链
router_chain = LLMRouterChain.from_llm(llm=online_llm, prompt=router_prompt, verbose=True, output_key="text")

In [ ]:
pip show langchain_core

Name: langchain-core
Version: 0.3.49
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /Users/zhangtian/miniforge3/envs/langchain/lib/python3.11/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing-extensions
Required-by: langchain, langchain-cohere, langchain-community, langchain-deepseek, langchain-experimental, langchain-huggingface, langchain-mcp-adapters, langchain-openai, langchain-text-splitters, langgraph, langgraph-checkpoint, langgraph-prebuilt
Note: you may need to restart the kernel to use updated packages.


In [181]:
#构建最终路由链，将路由链和默认链组合
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RouterRunnable

expert_chain = MultiPromptChain(
    tags=["output"],
    router_chain=router_chain, #路由链

    destination_chains={
        "math": math_LLMChain, #数学链
        "weather": weather_agent_text , #天气 智能体
        "time": time_agent_text , #时间 智能体
    }, #路由链对应的链

    default_chain=default_chain, #默认链
    verbose=True,
    
    # memory=memory, #记忆模块
)

In [182]:
#Test
result = expert_chain.invoke("明天是几号,北京的天气怎么样")
result



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
time: {'input': '明天是几号'}

> Entering new AgentExecutor chain...
Thought: 我需要查询当前日期，然后根据当前日期计算出明天的日期。

Action:
```
{
  "action": "get_date",
  "action_input": {}
}
```

Observation: 2025-05-28
Thought:根据当前日期2025-05-28，明天的日期应该是2025-05-29。

Action:
```
{
  "action": "Final Answer",
  "action_input": "明天是2025年5月29日"
}
```

> Finished chain.

> Finished chain.


{'input': '明天是几号', 'text': '明天是2025年5月29日'}

In [183]:
result = expert_chain.invoke("北京的天气怎么样")
result



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
weather: {'input': '北京的天气怎么样'}

> Entering new AgentExecutor chain...
我需要通过调用 get_weather API 来获取北京的天气情况。
Action: get_weather  
Action Input: 北京  开始查询北京的天气信息
{'reason': '查询成功!', 'result': {'city': '北京', 'realtime': {'temperature': '26', 'humidity': '35', 'info': '晴', 'wid': '00', 'direct': '东南风', 'power': '1级', 'aqi': '99'}, 'future': [{'date': '2025-05-28', 'temperature': '19/33℃', 'weather': '晴', 'wid': {'day': '00', 'night': '00'}, 'direct': '南风转西南风'}, {'date': '2025-05-29', 'temperature': '21/33℃', 'weather': '晴', 'wid': {'day': '00', 'night': '00'}, 'direct': '西南风转南风'}, {'date': '2025-05-30', 'temperature': '22/33℃', 'weather': '多云', 'wid': {'day': '01', 'night': '01'}, 'direct': '南风'}, {'date': '2025-05-31', 'temperature': '22/30℃', 'weather': '阴', 'wid': {'day': '02', 'night': '02'}, 'direct': '南风转东风'}, {'date': '2025-06-01', 'temperature': '22/30℃', 'weather': '阴转多云', 'wid': {

{'input': '北京的天气怎么样',
 'text': '北京当前的天气为晴，温度为26℃，湿度为35%，风向为东南风，风力等级为1级，空气质量指数（AQI）为99。未来几天的天气预报如下：\n\n- 2025年5月28日：晴，温度范围19至33℃，风向为南风转西南风。\n- 2025年5月29日：晴，温度范围21至33℃，风向为西南风转南风。\n- 2025年5月30日：多云，温度范围22至33℃，风向为南风。\n- 2025年5月31日：阴，温度范围22至30℃，风向为南风转东风。\n- 2025年6月1日：阴转多云，温度范围22至30℃，风向为东南风转东风。'}

In [184]:
result = expert_chain.invoke("榴莲是什么")
result



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '榴莲是什么'}
> Finished chain.


{'input': '榴莲是什么',
 'text': '榴莲是一种热带水果，素有“水果之王”的美誉。它原产于东南亚地区，如泰国、马来西亚和印度尼西亚等地，如今在中国的海南、广东、广西等南方地区也有种植。\n\n### 榴莲的主要特点：\n1. **外观**：榴莲的外壳坚硬，呈绿色或棕色，表面布满尖刺，形状多为椭圆形或卵形。\n2. **气味**：榴莲以其独特的气味闻名，有些人觉得香味浓郁诱人，而另一些人则认为气味刺鼻甚至难以接受。\n3. **果肉**：榴莲的果肉金黄色，质地柔软细腻，味道香甜，带有一丝奶油般的口感，有些品种还略带苦味或酸味。\n4. **种子**：榴莲内部含有几颗大种子，这些种子可以煮熟后食用，口感类似栗子。\n\n### 常见品种：\n- **猫山王（Musang King）**：产自马来西亚，果肉纤维少，口感细腻，甜中带苦。\n- **金枕（Monthong）**：泰国的主要出口品种，果肉较多，味道偏甜。\n- **甲仑（Jaren）**：果肉厚实，香味浓郁。\n\n### 营养价值：\n榴莲富含碳水化合物、蛋白质、脂肪以及多种维生素和矿物质，如维生素C、钾和铁。适量食用可以补充能量、增强免疫力，但因其热量较高，不宜过量食用。\n\n### 注意事项：\n1. **过敏反应**：部分人可能对榴莲过敏，食用时需注意。\n2. **保存方式**：未成熟的榴莲可放在阴凉处催熟，成熟后的榴莲最好冷藏保存，并尽快食用。\n3. **禁忌搭配**：榴莲性热，不宜与酒精一起食用，否则可能导致身体不适。\n\n榴莲的独特风味使其成为许多人喜爱的水果，但也因其气味强烈而备受争议。无论如何，它都是热带地区不可忽视的一种美味水果！'}